## Множественная проверка гипотез

In [1]:
import numpy as np
import pandas as pd
import itertools

from scipy import stats
from statsmodels.sandbox.stats.multicomp import multipletests 

### Question 2

Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. 

Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо.

In [2]:
data = pd.read_csv('AUCs.txt', sep='\t')
data.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


In [3]:
classificator_names = data.columns.values[1:]
combinations = itertools.combinations(classificator_names, 2)

p_values = np.array([])

for combination in combinations:
    stat = stats.wilcoxon(data.loc[:, combination[0]], data.loc[:, combination[1]])
    p_values = np.append(p_values, stat.pvalue)
    print "p-value for ", combination[0], " and ", combination[1], " classificators: ", stat.pvalue

p-value for  C4.5  and  C4.5+m  classificators:  0.0107571331198
p-value for  C4.5  and  C4.5+cf  classificators:  0.861262330095
p-value for  C4.5  and  C4.5+m+cf  classificators:  0.0159064441017
p-value for  C4.5+m  and  C4.5+cf  classificators:  0.0463327297934
p-value for  C4.5+m  and  C4.5+m+cf  classificators:  0.327825675845
p-value for  C4.5+cf  and  C4.5+m+cf  classificators:  0.0229090993544


**Ответ:** C4.5 and C4.5+m

### Question 3

Сколько статистически значимых на уровне 0.05 различий мы обнаружили?

In [5]:
print len(p_values[p_values < 0.05])

4


### Question 4

Судя по данным из предыдущего опроса, настройка какого из параметров классификатора даёт более значимое увеличение качества?

**Ответ:** m

### Question 5

Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [6]:
reject, p_corrected, a1, a2 = multipletests(p_values, alpha = 0.05,  method = 'holm')

print "corrected p-values: ", p_corrected
print sum(reject)

corrected p-values:  [ 0.0645428   0.86126233  0.07953222  0.13899819  0.65565135  0.0916364 ]
0


### Question 6

Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга?

In [7]:
reject, p_corrected, a1, a2 = multipletests(p_values, alpha = 0.05,  method = 'fdr_bh')

print "corrected p-values: ", p_corrected
print sum(reject)

corrected p-values:  [ 0.0458182   0.86126233  0.0458182   0.06949909  0.39339081  0.0458182 ]
3


### Question 7 

Насколько корректно, на ваш взгляд, применение метода Бенджамини-Хохберга в этой задаче?

**Ответ: ** Некорректно: статистики, соответствующие разным гипотезам, зависимы, так что предположения метода Бенджамини-Хохберга не выполняются